In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
%cd Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\SCOPA_Sex_Processed&Results_Files
data = pd.read_csv('SexFemaleCateg_Methylome_APPRDX.csv')
data.head(1)

Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\SCOPA_Sex_Processed&Results_Files


,Sentrix,PATNO,HYS,Gender,APPRDX,SCOPA_Sex,cg06168319,cg20161984,cg03497431,cg19050973,...,cg08830117,cg00289134,cg18304052,cg22552004,cg16216706,cg21146019,cg06720722,cg26251795,cg15535628,cg04860563
0,200991620021_R03C01,3002,2,2,1,1,0.011066,0.178036,0.01536,0.026076,...,0.844128,0.884618,0.87731,0.850813,0.751936,0.834262,0.782225,0.938638,0.902261,0.925377


In [3]:
X = data.drop(['PATNO','Sentrix','Gender','SCOPA_Sex', 'HYS'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 

In [4]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [5]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.8666666666666667

In [6]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100
Iteration: 33 / 100
Iteration: 34 / 100
Iteration: 35 / 100
Iteration: 36 / 100
Iteration: 37 / 100
Iteration: 38 / 100
Iteration: 39 / 100
Iteration: 40 / 100
Iteration: 41 / 100
Iteration: 42 / 100
Iteration: 43 / 100
Iteration: 44 / 100
Iteration: 45 / 100
Iteration: 46 / 100
Iteration: 47 / 100
Iteration: 48 / 100
Iteration: 49 / 100
Iteration: 50 / 100
Iteration

BorutaPy(estimator=RandomForestClassifier(n_estimators=34,
                                          random_state=RandomState(MT19937) at 0x231B4C3B740),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x231B4C3B740, verbose=1)

In [7]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  4


In [8]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [9]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head()

,CpG,Ranking
0,APPRDX,1
112,cg14773441,1
266,cg12061734,1
304,cg10769416,1
205,cg07584450,2


In [10]:
Top50.to_csv ('SexFemale_Boruta_Top50.csv', index = False)
selected_rf_features.to_csv ('SexFemale_Boruta_All.csv', index = False)